In [1]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch
from collections import defaultdict

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch
import shapely.wkt as wkt
from tqdm import tqdm
import fiona
import os

import alphashape
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from shapely.geometry import Point
import random

# result_df_name = 'sim_output_1pm_capacity_reduction_10k_PRELIMINARY'
result_df_name = 'sim_output_1pm_capacity_reduction_10k_test'
result_path = '../../../../data/datasets_simulation_outputs/' + result_df_name
string_is_for_1pm = "pop_1pm"

base_dir_sample_sim_input = '../../../../data/' + string_is_for_1pm + '_simulations/' + string_is_for_1pm + '_policies_combinations_with_normal_dist/'
subdirs_pattern = os.path.join(base_dir_sample_sim_input, 'output_networks_*')
subdirs = list(set(glob.glob(subdirs_pattern)))
subdirs.sort()

gdf_basecase_output_links = gpd.read_file('results/' + string_is_for_1pm + '_basecase_average_output_links.geojson')
gdf_basecase_average_mode_stats = pd.read_csv('results/' + string_is_for_1pm + '_basecase_average_mode_stats.csv', delimiter=';')
districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

## Abstract

This is further than process_output_of_simulations_with_all_output_links_and_eqasim_info.ipynb, as it also includes more input information.

Note that there is more than one strategy to deal with the fact that there are more than one district per link. We implement the strategy of stacking the information of all districts together. 
An alternative strategy would be to use the mean of the information of the districts.

Process the districts manually, so that each link belongs to at most 3 districts.

## Process results

Process the outputs of the simulations for further usage by GNN.

In [2]:
# Read all network data into a dictionary of GeoDataFrames
def compute_result_dic():
    result_dic_output_links = {}
    result_dic_eqasim_trips = {}
    base_network_no_policies = gdf_basecase_output_links
    result_dic_output_links["base_network_no_policies"] = base_network_no_policies
    counter = 0
    for subdir in tqdm(subdirs, desc="Processing subdirs", unit="subdir"):
        counter += 1
        if counter > 5:
            break
        # print(f'Accessing folder: {subdir}')
        # print(len(os.listdir(subdir)))
        networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
        for network in networks:
            file_path = os.path.join(subdir, network)
            policy_key = pio.create_policy_key_1pm(network)
            df_output_links = pio.read_output_links(file_path)
            df_eqasim_trips = pio.read_eqasim_trips(file_path)
            if (df_output_links is not None and df_eqasim_trips is not None):
                df_output_links.drop(columns=['geometry'], inplace=True)
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=df_output_links, column_to_extend='highway', new_column_name='highway')
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
                result_dic_output_links[policy_key] = gdf_extended
                mode_stats = pio.calculate_averaged_results(df_eqasim_trips)
                result_dic_eqasim_trips[policy_key] = mode_stats
    return result_dic_output_links, result_dic_eqasim_trips

result_dic_output_links, result_dic_eqasim_trips = compute_result_dic()
print(len(result_dic_eqasim_trips.keys()))
print(len(result_dic_output_links.keys()))

base_gdf = result_dic_output_links["base_network_no_policies"]
links_gdf_base = gpd.GeoDataFrame(base_gdf, geometry='geometry')
links_gdf_base.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
links_gdf_base.to_crs("EPSG:4326", inplace=True)
districts['district_centroid'] = districts['geometry'].centroid
links_gdf_with_districts = gpd.sjoin(links_gdf_base, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
links_gdf_with_districts = links_gdf_with_districts.groupby('link').agg({
    'from_node': 'first',
    'to_node': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'lanes': 'first',
    'modes': 'first',
    'vol_car': 'first',
    'highway': 'first',
    'geometry': 'first',
    'osm:way:oneway': 'first',
    'c_ar': lambda x: list(x.dropna()),
    'district_centroid': lambda x: list(x.dropna()),
    'perimetre': lambda x: list(x.dropna()),
    'surface': lambda x: list(x.dropna()),
}).reset_index()

gdf_now = gpd.GeoDataFrame(links_gdf_with_districts, geometry='geometry', crs=links_gdf_base.crs)
gdf_now = gdf_now.rename(columns={'c_ar': 'district', 'perimetre': 'district_perimeter', 'surface': 'district_surface'})

links_gdf = pio.summarize_duplicate_edges(gdf_now)
print(f"Original number of edges: {len(gdf_now)}")
print(f"Number of edges after summarization: {len(links_gdf)}")
remaining_duplicates = pio.find_duplicate_edges_in_gdf(links_gdf)
print(f"Number of remaining duplicate edges: {len(remaining_duplicates)}")

Processing subdirs:   5%|▌         | 5/100 [00:45<14:16,  9.01s/subdir]


448
449


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_46626/1202570188.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  districts['district_centroid'] = districts['geometry'].centroid
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


Original number of edges: 31216
Number of edges after summarization: 25309
Number of remaining duplicate edges: 0


In [3]:
# PROCESS LINK GEOMETRIES
edge_start_point_tensor, stacked_edge_geometries_tensor, district_centroids_tensor_padded, edges_base, nodes = pio.get_link_geometries(links_gdf, districts)

In [8]:
links_gdf_base.head()

,link,from_node,to_node,length,freespeed,capacity,lanes,modes,vol_car,osm:relation:route_master,...,osm:way:id,osm:way:access,osm:way:oneway,highway,osm:relation:route,osm:way:railway,osm:way:name,storageCapacityUsedInQsim,osm:way:tunnel,geometry
0,100315,24972409,24972408,16.181257,8.333333,480.0,1.0,"bus,car,car_passenger",6.627451,NaN,...,4216830.0,NaN,yes,residential,bicycle,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (2.33869 48.85181, 2.33847 48.85181)"
1,100316,5904976363,24983651,14.860209,8.333333,480.0,1.0,"bus,car,car_passenger,pt",9.607843,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (2.33874 48.85242, 2.33872 48.85229)"
2,100317,24983651,5904976363,14.860209,8.333333,960.0,2.0,"bus,car,car_passenger,pt",2.490196,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (2.33872 48.85229, 2.33874 48.85242)"
3,100321,664205947,24972376,22.264540,8.333333,960.0,2.0,"car,car_passenger",6.941176,NaN,...,4216834.0,NaN,yes,residential,NaN,NaN,Boulevard Saint-Germain,NaN,NaN,"LINESTRING (2.33994 48.85200, 2.33986 48.85181)"
4,100324,24972376,24972375,64.853276,8.333333,480.0,1.0,"bus,car,car_passenger",7.607843,NaN,...,4216833.0,NaN,yes,residential,bicycle,NaN,Rue Dupuytren,NaN,NaN,"LINESTRING (2.33986 48.85181, 2.33909 48.85152)"


In [10]:
links_gdf_base['osm:way:oneway']
# Investigate the 'osm:way:oneway' column in links_gdf_base
oneway_counts_base = links_gdf_base['osm:way:oneway'].value_counts(dropna=False)
print("Counts in links_gdf_base['osm:way:oneway']:")
print(oneway_counts_base)

# Investigate the 'osm:way:oneway' column in links_gdf
oneway_counts = links_gdf['osm:way:oneway'].value_counts(dropna=False)
print("Counts in links_gdf['osm:way:oneway']:")
print(oneway_counts)

# Compare the counts
comparison_df = pd.DataFrame({
    'links_gdf_base': oneway_counts_base,
    'links_gdf': oneway_counts
}).fillna(0)
print("Comparison of 'osm:way:oneway' counts between links_gdf_base and links_gdf:")
print(comparison_df)

# Check for roads where "from_node" is equal to "to_node"
same_node_edges = links_gdf[links_gdf['from_node'] == links_gdf['to_node']]
print(f"Number of edges where 'from_node' is equal to 'to_node': {len(same_node_edges)}")

# Check for edges which exist in both directions
reverse_edges = links_gdf.merge(links_gdf, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
print(f"Number of edges which exist in both directions: {len(reverse_edges)}")

# Check for roads where "from_node" is equal to "to_node" in links_gdf_base
same_node_edges_base = links_gdf_base[links_gdf_base['from_node'] == links_gdf_base['to_node']]
print(f"Number of edges in links_gdf_base where 'from_node' is equal to 'to_node': {len(same_node_edges_base)}")

# Check for edges which exist in both directions in links_gdf_base
reverse_edges_base = links_gdf_base.merge(links_gdf_base, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
print(f"Number of edges in links_gdf_base which exist in both directions: {len(reverse_edges_base)}")

# Check for roads where "from_node" is equal to "to_node" in links_gdf
same_node_edges = links_gdf[links_gdf['from_node'] == links_gdf['to_node']]
print(f"Number of edges in links_gdf where 'from_node' is equal to 'to_node': {len(same_node_edges)}")

# Check for edges which exist in both directions in links_gdf
reverse_edges = links_gdf.merge(links_gdf, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
print(f"Number of edges in links_gdf which exist in both directions: {len(reverse_edges)}")



Counts in links_gdf_base['osm:way:oneway']:
yes    17589
NaN    12466
no      1148
-1        13
Name: osm:way:oneway, dtype: int64
Counts in links_gdf['osm:way:oneway']:
yes    17537
NaN     7154
no       610
-1         8
Name: osm:way:oneway, dtype: int64
Comparison of 'osm:way:oneway' counts between links_gdf_base and links_gdf:
     links_gdf_base  links_gdf
yes           17589      17537
NaN           12466       7154
no             1148        610
-1               13          8
Number of edges where 'from_node' is equal to 'to_node': 766
Number of edges which exist in both directions: 766
Number of edges in links_gdf_base where 'from_node' is equal to 'to_node': 769
Number of edges in links_gdf_base which exist in both directions: 12595
Number of edges in links_gdf where 'from_node' is equal to 'to_node': 766
Number of edges in links_gdf which exist in both directions: 766


# Analyze results and save to file

In [5]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [6]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [7]:
def process_result_dic(result_dic, result_dic_mode_stats, save_path=None, batch_size=500, gdf_input=None):
    os.makedirs(save_path, exist_ok=True)
    datalist = []
    linegraph_transformation = LineGraph()
    
    vol_base_case = links_gdf['vol_car'].values
    capacity_base_case = np.where(links_gdf['modes'].str.contains('car'), links_gdf['capacity'], 0)
    length = links_gdf['length'].values
    freespeed_base_case = links_gdf['freespeed'].values
    allowed_modes = pio.encode_modes(links_gdf)
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x)
    
    batch_counter = 0
    for key, df in tqdm(result_dic.items(), desc="Processing result_dic", unit="dataframe"):   
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = pio.prepare_gdf(df, gdf_input)
            len_edges = len(gdf)
            
            capacities_new, capacity_reduction, highway, freespeed =  pio.get_basic_edge_attributes(capacity_base_case, gdf)
            district_info, combined_tensor = pio.compute_combined_tensor(vol_base_case, capacity_base_case, length, freespeed_base_case, allowed_modes, gdf, capacities_new, capacity_reduction, highway, freespeed)
            
            linegraph_data = linegraph_transformation(data)
            linegraph_data.x = combined_tensor
            linegraph_data.num_nodes = combined_tensor.shape[0]
        
            # add edge attributes: 1 if edge to district, 0 if edge to edge
            edge_to_district_index, edge_to_district_attr = pio.compute_edge_attributes(district_info, linegraph_data, len_edges, gdf)
            if linegraph_data.edge_attr is None:
                linegraph_data.edge_attr = torch.zeros((linegraph_data.edge_index.shape[1] - edge_to_district_index.shape[1], 1), dtype=torch.long)
            linegraph_data.edge_attr = torch.cat([linegraph_data.edge_attr, edge_to_district_attr], dim=0)

            # add node attributes: 1 if district, 0 if edge
            node_type_feature = pio.compute_node_attributes(district_info, len_edges)
            linegraph_data.x = torch.cat([linegraph_data.x, node_type_feature], dim=1)
            linegraph_data.pos = torch.cat([stacked_edge_geometries_tensor, district_centroids_tensor_padded], dim=0)
            linegraph_data.y = pio.compute_target_tensor(vol_base_case, gdf, district_info)
                        
            df_mode_stats = result_dic_mode_stats.get(key)
            if df_mode_stats is not None:
                numeric_cols = df_mode_stats.select_dtypes(include=[np.number]).columns
                mode_stats_numeric = df_mode_stats[numeric_cols].astype(float)
                mode_stats_tensor = torch.tensor(mode_stats_numeric.values, dtype=torch.float)
                linegraph_data.mode_stats = mode_stats_tensor
            
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
                batch_counter += 1

                # Save intermediate result every batch_size data points
                if batch_counter % batch_size == 0:
                    batch_index = batch_counter // batch_size
                    torch.save(datalist, os.path.join(save_path, f'datalist_batch_{batch_index}.pt'))
                    datalist = []  # Reset datalist for the next batch
            else:
                print("Invalid line graph data")
    
    # Save any remaining data points
    if datalist:
        batch_index = (batch_counter // batch_size) + 1
        torch.save(datalist, os.path.join(save_path, f'datalist_batch_{batch_index}.pt'))
    return

# Call the function
process_result_dic(result_dic=result_dic_output_links, result_dic_mode_stats=result_dic_eqasim_trips, save_path=result_path, batch_size=200, gdf_input=links_gdf)

6


Processing result_dic: 100%|██████████| 449/449 [13:52<00:00,  1.86s/dataframe]
